# Download annual per capita nightlights 

Create a csv file with annual per capita nightlight values over DHS clusters, for years 2000 to 2013.

Import, authenticate and initialize the earth-engine library

In [1]:
import ee

ee.Authenticate()

ee.Initialize()

Enter verification code: 4/1Adeu5BXNYnz2Y_rzR5n_3z6iinngxIs-VRN382z3xx5MXsoV2lOwE9nQ4-A

Successfully saved authorization token.


In [22]:
# Import other libraries
import csv
import os
import pandas as pd

Read the csv file with survey points

In [29]:
data_dir = '/mimer/NOBACKUP/groups/globalpoverty1/cindy/eoml_ch_wb/data/interim'
dhs_cluster_file_path = os.path.join(data_dir, 'dhs_treat_control_confounders.csv')
df = pd.read_csv(dhs_cluster_file_path)
unique_countries = df['country'].unique()
df.head()

,country,survey_start_year,year,lat,lon,households,rural,iwi,dhs_id,image_file,...,log_trans_proj_cum_n_2004,log_trans_proj_cum_n_2005,log_trans_proj_cum_n_2006,log_trans_proj_cum_n_2007,log_trans_proj_cum_n_2008,log_trans_proj_cum_n_2009,log_trans_proj_cum_n_2010,log_trans_proj_cum_n_2011,log_trans_proj_cum_n_2012,log_trans_proj_cum_n_2013
0,south_africa,2016,2016,-34.463232,19.542468,6,1,70.723295,48830,./data/dhs_tifs/south_africa_2016/00743.tif,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,south_africa,2016,2016,-34.418873,19.188926,11,0,76.798705,48781,./data/dhs_tifs/south_africa_2016/00694.tif,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,south_africa,2016,2016,-34.412835,19.178965,4,0,81.053723,48828,./data/dhs_tifs/south_africa_2016/00741.tif,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,south_africa,2016,2016,-34.292107,19.563813,6,1,72.766880,48787,./data/dhs_tifs/south_africa_2016/00700.tif,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,south_africa,2016,2016,-34.187500,22.113079,3,0,77.864113,48756,./data/dhs_tifs/south_africa_2016/00669.tif,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Function to create a feature with a 6.7km square around a lat/lon

In [105]:
def createDHSFeature(row):
  lat = ee.Number(row['lat'])
  lon = ee.Number(row['lon'])
  dhs_id = ee.Number(row['dhs_id'])
  properties = {'dhs_id':dhs_id,
               'lat':lat,
               'lon':lon}
    
  #Calc coordinates of a 6.7km bounding box around the point
  feature_radius=3350
  roi = ee.Geometry.Point(lon,lat).buffer(feature_radius).bounds()
  
  return ee.Feature(roi, properties)

Load nightlight and population density collections over the African continent

In [114]:
africa_region = ee.Geometry.Polygon([
    [-20, 38],   
    [55, 38],    
    [55, -36],   
    [-20, -36]   
])

dmsp = ee.ImageCollection('NOAA/DMSP-OLS/NIGHTTIME_LIGHTS').filterBounds(africa_region)
pop_count = ee.ImageCollection(ee.ImageCollection("CIESIN/GPWv411/GPW_Population_Count")).filterBounds(africa_region)

#get a composite image for each nighlight year and store in a dictionary
dmsp_images = {}
for year in range(2000, 2014):
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    
    # Construct the variable name
    variable_name = f"dmsp{year}img"
    img = dmsp.filterDate(start_date, end_date).select(["stable_lights"]).median()
    
    # Create the image using the filterDate and select operations
    dmsp_images[year] = {'img':img,'year':year}
    
#loop makes code below unnecessary
# dmsp2000img = dmsp.filterDate('2000-01-01', '2000-12-31').select(["stable_lights"]).median()
# dmsp2001img = dmsp.filterDate('2001-01-01', '2001-12-31').select(["stable_lights"]).median()
# dmsp2002img = dmsp.filterDate('2002-01-01', '2002-12-31').select(["stable_lights"]).median()
# dmsp2003img = dmsp.filterDate('2003-01-01', '2003-12-31').select(["stable_lights"]).median()
# dmsp2004img = dmsp.filterDate('2004-01-01', '2004-12-31').select(["stable_lights"]).median()
# dmsp2005img = dmsp.filterDate('2005-01-01', '2005-12-31').select(["stable_lights"]).median()
# dmsp2006img = dmsp.filterDate('2006-01-01', '2006-12-31').select(["stable_lights"]).median()
# dmsp2007img = dmsp.filterDate('2007-01-01', '2007-12-31').select(["stable_lights"]).median()
# dmsp2008img = dmsp.filterDate('2008-01-01', '2008-12-31').select(["stable_lights"]).median()
# dmsp2009img = dmsp.filterDate('2009-01-01', '2009-12-31').select(["stable_lights"]).median()
# dmsp2010img = dmsp.filterDate('2010-01-01', '2010-12-31').select(["stable_lights"]).median()
# dmsp2011img = dmsp.filterDate('2011-01-01', '2011-12-31').select(["stable_lights"]).median()
# dmsp2012img = dmsp.filterDate('2012-01-01', '2012-12-31').select(["stable_lights"]).median()
# dmsp2013img = dmsp.filterDate('2013-01-01', '2013-12-31').select(["stable_lights"]).median()


pop_count_images = {}
years_to_extract = [2000, 2005, 2010]  

for year in years_to_extract:
    start_date = f"{year}-01-01"
    end_date = f"{year}-12-31"
    
    # Construct the variable name
    variable_name = f"pop_count{year}img"
    
    # Create the image using the filterDate operation
    img = pop_count.filterDate(start_date, end_date).select(["population_count"]).first()
    
    # Store the image using the variable name
    pop_count_images[year] = {'img':img,'year':year}

#Loop makes code below unnecessary
# pop_count2000img = pop_count.filterDate('2000-01-01', '2000-12-31').select(["population_count"]).first()
# pop_count2005img = pop_count.filterDate('2005-01-01', '2005-12-31').select(["population_count"]).first()
# pop_count2010img = pop_count.filterDate('2010-01-01', '2010-12-31').select(["population_count"]).first()

Function to calculate sum of nightlights within a 6.7km2 square

In [115]:
def calculate_nl_sums(feature):
    return_feature_set = {}   #initialize empty dictionary to return
    
    #calculate annual nighlight sums
    for year, data in dmsp_images.items():
        img = data['img']
        year_value = data['year']

        nlyear = img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=feature.geometry(),
            scale=927.67
        )
        sum_nlyear = nlyear.getNumber('stable_lights')
        rounded_nlyear = ee.Number(sum_nlyear).format('%.1f')

        #append this year to the feature set
        prop_name = f'nl{year_value}'
        return_feature_set[prop_name] = rounded_nlyear 
     
    #calculate 5-year pop count sums
#     for year, data in pop_count_images.items():
#         img = data['img']
#         year_value = data['year']

#         pcountyear = img.reduceRegion(
#             reducer=ee.Reducer.sum(),
#             geometry=feature.geometry(),
#             scale=927.67
#         )
#         sum_pcountyear = pcountyear.getNumber('population_count')
#         rounded_sum_pcountyear = ee.Number(sum_pcountyear).format('%.1f')

#         #append this year to the feature set
#         prop_name = f'pop_count{year_value}'
#         return_feature_set[prop_name] = rounded_sum_pcountyear  
    
    return feature.set(return_feature_set)

In [116]:
def calculate_popcount_sums(feature):
    return_feature_set = {}   #initialize empty dictionary to return
        
    #calculate 5-year pop count sums
    for year, data in pop_count_images.items():
        img = data['img']
        year_value = data['year']

        pcountyear = img.reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=feature.geometry(),
            scale=927.67
        )
        sum_pcountyear = pcountyear.getNumber('population_count')
        rounded_sum_pcountyear = ee.Number(sum_pcountyear).format('%.1f')

        #append this year to the feature set
        prop_name = f'pop_count{year_value}'
        return_feature_set[prop_name] = rounded_sum_pcountyear  
    
    return feature.set(return_feature_set)

Open csv file, write header, loop over years and DHS points

In [ ]:
for country in unique_countries:
    country_df = df[df['country'] == country]
    print(f"Country: {country}")

    # create list of features from the country dataframe
    feature_list = [createDHSFeature(row) for index, row in country_df.iterrows()]

    # Create a Feature Collection from the list of features
    country_collection = ee.FeatureCollection(feature_list)
 
    # calculate nightlights and population count sums
    country_collection2 = country_collection.map(calculate_nl_sums)
    country_collection3 = country_collection2.map(calculate_popcount_sums)
    
    
# Write to CSV file 
header = ['dhs_id', 'lat', 'lon', 'nl2000', 'nl2001', 'nl2002'
         'nl2003','nl2004','nl2005','nl2006','nl2007','nl2008',
          'nl2009','nl2010','nl2011','nl2012','nl2013','pop_count2000',
         'pop_count2005','pop_count2010']
         
# Create a list to store the rows of data
rows = []

# Loop through the features in the collection and extract the data
for feature in country_collection3.getInfo()['features']:
    properties = feature['properties']
    row = [
        properties['dhs_id'],
        properties['lat'],
        properties['lon'],
        properties['nl2000'],
        properties['nl2001'],
        properties['nl2002'],
        properties['nl2003'],
        properties['nl2004'],
        properties['nl2005'],
        properties['nl2006'],
        properties['nl2007'],
        properties['nl2008'],
        properties['nl2009'],
        properties['nl2010'],
        properties['nl2011'],
        properties['nl2012'],
        properties['nl2013'],
        properties['pop_count2000'],
        properties['pop_count2005'],
        properties['pop_count2010']      
    ]
    rows.append(row)

# Specify the name of the CSV file
csv_file_path = '/mimer/NOBACKUP/groups/globalpoverty1/cindy/eoml_ch_wb/data/per_cap_nl_dhs.csv'

# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write the header
    csv_writer.writerow(header)
    
    # Write the rows
    csv_writer.writerows(rows)

print("CSV file has been successfully written.")

   



Country: south_africa
Country: lesotho
Country: namibia
Country: eswatini
Country: mozambique
Country: madagascar
Country: zimbabwe
Country: zambia
Country: malawi
Country: angola
Country: comoros
Country: democratic_republic_of_congo
Country: tanzania
Country: kenya
Country: burundi
Country: gabon
Country: rwanda
Country: uganda
Country: cameroon
Country: ethiopia
Country: central_african_republic
Country: nigeria
Country: ghana
Country: ivory_coast
Country: liberia
Country: togo
Country: benin
Country: sierra_leone
Country: guinea
Country: chad
Country: burkina_faso
Country: mali
Country: niger
Country: senegal
Country: egypt
Country: morocco
